In [ ]:
import pandas as pd
import numpy as np
import random

random.seed(123)
np.random.seed(123)

In [ ]:
df_imp = pd.read_csv("FL_hurricane_county_year_impacts.csv")
df_home = pd.read_csv("FL_home_values.csv")
df_exp = pd.read_csv("FL_hurricane_exposures.csv")
df_nhs = pd.read_csv("FL_NHS.csv")
df_ipums = pd.read_csv("FL_IPUMS.csv")
df_claims = pd.read_csv("FL_NFIP_Claims.csv")
df_policy = pd.read_csv("FL_policy&loss_statistics.csv")
df_homeval = pd.read_csv("FL_house_prices_new2.csv")

In [ ]:
def filter_county(df, columns, county_name, label="", nearby=None):
    """Filter a dataframe for a given county or its nearby alternatives."""
    county_df = pd.DataFrame()

    for col in columns:
        if col in df.columns:
            # Try flexible matching for names like "Duval" or "Duval County"
            mask = df[col].astype(str).str.contains(county_name, case=False, na=False)
            county_df = df[mask]
            if not county_df.empty:
                print(f"{label}: Using {county_name} County data.")
                return county_df

            # Try nearby counties if nothing found
            if nearby and county_name in nearby:
                for near in nearby[county_name]:
                    mask_near = df[col].astype(str).str.contains(near, case=False, na=False)
                    county_df = df[mask_near]
                    if not county_df.empty:
                        print(f"{label}: No {county_name} data — using {near} County instead.")
                        return county_df
    return pd.DataFrame()

In [ ]:
def fallback_if_empty(df, label, nearby_dfs=None, global_df=None):
    """
    Returns the original DataFrame if not empty.
    Otherwise, tries nearby DataFrames or a random sample from the global dataset.
    """
    if not df.empty:
        return df

    # Try nearby datasets if given
    if nearby_dfs:
        for n_df, n_name in nearby_dfs:
            if not n_df.empty:
                print(f"⚠️ {label}: Using nearby county ({n_name}) as fallback.")
                return n_df

    # Try random sample from global dataset
    if global_df is not None and not global_df.empty:
        print(f"⚠️ {label}: No nearby data — using random sample from dataset.")
        return global_df.sample(frac=0.3, random_state=42)  # use 30% of full data

    print(f"⚠️ {label}: No data available at all.")
    return pd.DataFrame()


In [ ]:
def build_persona(county_name, nearby=None, mode = 'base'):
    if nearby is None:
        nearby = []

    # Map county names to their FIPS suffix (from IPUMS or other data sources)
    county_fips_map = {
        "Duval": "031",
        "Broward": "011",
        "Alachua": "001",
        "St. Johns": "109",
        "Leon": "073",
        "Bay": "005",
        "Brevard": "009",
        "Pasco": "101",
        "Seminole": "117",
        "Palm Beach": "099",
        "Osceola": "097",
        "Hillsborough": "057",
        "Pinellas": "103",
        "Manatee": "081",
        "Sarasota": "115",
        "Lee": "071",
        "Clay": "019",
        "Nassau": "089",
        "Baker": "003",
        "Flagler": "035",
        "Putnam": "107",
        "Gadsden": "039",
        "Wakulla": "129",
        "Calhoun": "013",
        "Gulf": "045",
        "Walton": "131",
        "Indian River": "061",
        "Orange": "095",
        "Hernando": "053",
        "Charlotte": "015",
        "Collier": "021",
        "DeSoto": "027",
    }

    # Get the FIPS suffix
    fips_suffix = county_fips_map.get(county_name, "")
    if not fips_suffix:
        print(f"⚠️ No FIPS code found for {county_name}. Using nearby dataset as fallback.")

    print(f"\n📊 Building persona for {county_name} (FIPS suffix {fips_suffix})")

    # Filter each dataset
    county_imp = df_imp[df_imp["fips"].astype(str).str.endswith(fips_suffix)]
    county_home = filter_county(df_home, ["RegionName"], county_name, label="Home", nearby=nearby)
    county_exp = filter_county(df_exp, ["county", "County"], county_name, label="Exposure", nearby=nearby)
    county_nhs = filter_county(df_nhs, ["county", "County"], county_name, label="NHS", nearby=nearby)
    county_ipums = df_ipums[df_ipums["COUNTYFIP"].astype(str).str.zfill(3).str.endswith(fips_suffix)]
    county_nfip = df_claims[df_claims["county_code"].astype(str).str.endswith(fips_suffix)]
    county_homeval = filter_county(df_homeval, ["county", "County"], county_name, label="HomeVal", nearby=nearby)
    # Fallback if empty (uses nearby or random sample)
    county_imp = fallback_if_empty(county_imp, "IMP Data", global_df=df_imp)
    county_home = fallback_if_empty(county_home, "Home Data", global_df=df_home)
    county_exp = fallback_if_empty(county_exp, "Exposure Data", global_df=df_exp)
    county_nhs = fallback_if_empty(county_nhs, "NHS Data", global_df=df_nhs)
    county_ipums = fallback_if_empty(county_ipums, "IPUMS Data", global_df=df_ipums)
    county_nfip = fallback_if_empty(county_nfip, "NFIP Data", global_df=df_claims)
    county_homeval = fallback_if_empty(county_homeval, "HomeVal Data", global_df=df_homeval)
    # Return the persona DataFrame
    return create_persona(county_name, county_imp, county_home, county_exp, county_nhs, county_ipums, county_nfip, df_policy, county_homeval,  mode = mode)

In [ ]:
def create_persona(county_name, df_imp, df_home, df_exp, df_nhs, df_ipums, df_nfip, df_policy, df_homeval, mode='base'):
    education_order = [
        "Less than high school diploma",
        "High school degree or diploma",
        "Some college, no degree",
        "Associate's degree",
        "Bachelor's degree",
        "Post graduate work/degree or professional degree"
    ]
    if not df_nhs.empty:
        df_nhs = df_nhs.copy()
        df_nhs["education"] = pd.Categorical(df_nhs["education"], categories=education_order, ordered=True)

    risk_order = ["Unlikely", "Don't Know", "Likely", "Very likely"]

    if not df_nhs.empty:
        df_nhs["hrcn_risk_perception"] = pd.Categorical(df_nhs["hrcn_risk_perception"],categories=risk_order, ordered=True)

    if mode == "random":
        age = 19
        income_median = np.nan
        home_value = np.nan
        wind_avg = np.nan
        storm_count = 0
        education = "Unknown"
        preparedness = "Unknown"
        risk_response = "Unknown"
        kids = 0
        employment = "unknown"
        status = "Unknown"
        flood_zone = "Unknown"
        elevation_diff = "Unknown"
        flood_depth = "Unknown"
        construction_year = "Unknown"

        # EXPOSURE data
        if not df_exp.empty:
            exp_sample = df_exp.sample(1, replace=True).iloc[0]
            wind_avg = exp_sample.get('max_wind_speed', np.nan)
            storm_count = df_exp['hurricane_name'].nunique() if 'hurricane_name' in df_exp.columns else 0

        # NHS data
        if not df_nhs.empty:
            nhs_sample = df_nhs.sample(1, replace=True).iloc[0]
            education = nhs_sample.get('education', 'Unknown')
            preparedness = nhs_sample.get('hrcn_prep_3cat', 'Unknown')
            risk_response = nhs_sample.get('hrcn_risk_perception', 'Unknown')
            kids = nhs_sample.get('numchild', 0)
        else:
            education = preparedness = risk_response = 'Unknown'
            kids = 0

        # IPUMS data
        if not df_ipums.empty:
            adults = df_ipums[df_ipums["AGE"] >= 18].copy()
            if not adults.empty:
                ipums_sample = adults.sample(1, replace=True).iloc[0]
                valid_income = df_ipums.loc[(df_ipums["INCTOT"] > 0) & (~df_ipums["INCTOT"].isna()), "INCTOT"]
                if not valid_income.empty:
                    income_median = int(valid_income.sample(1).iloc[0])
            else:
                ipums_sample = df_ipums.sample(1, replace=True).iloc[0]

            emp = ipums_sample.get('EMPSTAT', np.nan)

            if emp == 1:
                employment = "employed"
            elif (emp == 2 or income_median == None or pd.isna(income_median)) and age <= 60:
                employment = "unemployed"
            elif emp == 3:
                employment = "not in labor force"
            else:
                employment = "unknown"

            m = ipums_sample.get('MARST', np.nan)
            status = {
                1: "Married with spouse present",
                2: "Married with spouse absent",
                3: "Separated",
                4: "Divorced",
                5: "Widowed",
            }.get(m, "Single")

        # NFIP data
        if not df_nfip.empty:
            nfip_sample = df_nfip.sample(1, replace=True).iloc[0]
            flood_zone = nfip_sample.get("flood_zone", "Unknown")
            elevation_diff = nfip_sample.get("elevation_difference_ft", "Unknown")
            flood_depth = nfip_sample.get("water_depth_ft", "Unknown")
            construction_year = nfip_sample.get("construction_year", "Unknown")
        else:
            flood_zone = elevation_diff = flood_depth = construction_year = "Unknown"

        # POLICY data - calculate premium
        try:
            avg_premium = (
                df_policy["Total Annual Payment"].sum() /
                df_policy["Policies in Force"].sum()
            )
            avg_premium = round(avg_premium)
        except (KeyError, ZeroDivisionError):
            avg_premium = 1500

#----------------------------------------------------------------------------------------------------------------------------#
    elif mode == "max":

        if not df_ipums.empty and "AGE" in df_ipums.columns:
            adults = df_ipums[df_ipums["AGE"] >= 18]

        if "INCTOT" in df_ipums.columns and not df_ipums.empty:
            valid_income = adults.loc[adults["INCTOT"] > 0, "INCTOT"] if not adults.empty else pd.Series()
            income_median = int(valid_income.max()) if not valid_income.empty else np.nan
        else:
            income_median = np.nan

    # Employment (always employed for "max")
        employment = "employed"

    # --- Education ---
        if not df_nhs.empty and "education" in df_nhs.columns:
            education = df_nhs["education"].dropna().max()
        else:
            education = "Post graduate work/degree or professional degree"

    # --- Preparedness ---
        if not df_nhs.empty and "hrcn_prep_3cat" in df_nhs.columns:
            preparedness = df_nhs["hrcn_risk_perception"].dropna().max()
        else:
            preparedness = "Very Likely"

        status = "Married with spouse present"

        kids = round(df_nhs["numchild"].max()) if "numchild" in df_nhs.columns and not df_nhs.empty else 0

        if not df_nfip.empty and "water_depth_ft" in df_nfip.columns:
            valid_depths = df_nfip["water_depth_ft"].dropna()
            if not valid_depths.empty:
                row = df_nfip.loc[valid_depths.idxmin()]
            else:
                row = df_nfip.sample(1).iloc[0]  # fallback to any random row
        else:
            row = None
        if row is not None:
            flood_zone = row.get("flood_zone", "Unknown")
            elevation_diff = row.get("elevation_difference_ft", "Unknown")
            flood_depth = row.get("water_depth_ft", "Unknown")
            construction_year = row.get("construction_year", "Unknown")
        else:
            flood_zone = elevation_diff = flood_depth = construction_year = "Unknown"

        # Hurricane data
        wind_avg = round(df_exp["max_wind_speed"].max(),1) if not df_exp.empty else np.nan
        storm_count = df_exp["hurricane_name"].nunique() if not df_exp.empty else 0

        # Risk perception
        if "hrcn_risk_perception" in df_nhs.columns and not df_nhs.empty:
            risk_response = df_nhs["hrcn_risk_perception"].dropna().max()
        else:
            risk_response = "Very Likely"

        try:
            avg_premium = (
                df_policy["Total Annual Payment"].sum() /
                df_policy["Policies in Force"].sum()
            )
            avg_premium = round(avg_premium)
        except (KeyError, ZeroDivisionError):
            avg_premium = 1500

#---------------------------------------------------------------------------------------------------------------------------------------------------#

    elif mode == 'min':

        if not df_ipums.empty and "AGE" in df_ipums.columns:
            adults = df_ipums[df_ipums["AGE"] >= 18]

        if "INCTOT" in df_ipums.columns and not df_ipums.empty:
            valid_income = adults.loc[adults["INCTOT"] > 0, "INCTOT"] if not adults.empty else pd.Series()
            income_median = int(valid_income.min()) if not valid_income.empty else np.nan
        else:
            income_median = 0

        employment = "unemployed"

    # --- Education ---
        if not df_nhs.empty and "education" in df_nhs.columns:
            education = df_nhs["education"].dropna().min()
        else:
            education = "Unknown"

    # --- Preparedness ---
        if not df_nhs.empty and "hrcn_prep_3cat" in df_nhs.columns:
            preparedness = df_nhs["hrcn_prep_3cat"].dropna().min()
        else:
            preparedness = "Unknown"

    # --- Marital Status ---
        status = "Single"

        # Number of kids
        kids = round(df_nhs["numchild"].min()) if "numchild" in df_nhs.columns and not df_nhs.empty else 0

        # Flood data
        if not df_nfip.empty and "water_depth_ft" in df_nfip.columns:
            # Drop NaNs and make sure there’s something left
            valid_depths = df_nfip["water_depth_ft"].dropna()
            if not valid_depths.empty:
                row = df_nfip.loc[valid_depths.idxmin()]
            else:
                row = df_nfip.sample(1).iloc[0]  # fallback to any random row
        else:
            row = None
        if row is not None:
            flood_zone = row.get("flood_zone", "Unknown")
            elevation_diff = row.get("elevation_difference_ft", "Unknown")
            flood_depth = row.get("water_depth_ft", "Unknown")
            construction_year = row.get("construction_year", "Unknown")
        else:
            flood_zone = elevation_diff = flood_depth = construction_year = "Unknown"

        # Hurricane data
        wind_avg = round(df_exp["max_wind_speed"].min(),1) if not df_exp.empty else np.nan
        storm_count = df_exp["hurricane_name"].nunique() if not df_exp.empty else 0

        # Risk perception
        if "hrcn_risk_perception" in df_nhs.columns and not df_nhs.empty:
            risk_response = df_nhs["hrcn_risk_perception"].dropna().min()
        else:
            risk_response = "Very unlikely"

        #this you can ignore forever
        try:
            avg_premium = (
                df_policy["Total Annual Payment"].sum() /
                df_policy["Policies in Force"].sum()
            )
            avg_premium = round(avg_premium)
        except (KeyError, ZeroDivisionError):
            avg_premium = 1500
#-------------------------------------------------------------------------------------------------------------------------------------#
    else:
        # this is where everything is based at some baseline values
        # IPUMS: demographics
        if not df_ipums.empty and "AGE" in df_ipums.columns:
            adults = df_ipums[df_ipums["AGE"] >= 18]

        if "INCTOT" in df_ipums.columns:
            # Filter out invalid/missing income values if needed
            valid_income = adults.loc[adults["INCTOT"] > 0, "INCTOT"]
            income_median = int(valid_income.median()) if not valid_income.empty else np.nan

        # Employment

        if "EMPSTAT" in df_ipums.columns and not df_ipums.empty:
            emp = df_ipums["EMPSTAT"].mode()[0]
        else:
            emp = np.nan

        if emp == 1:
            employment = "employed"
        elif (emp == 2 or income_median == None or pd.isna(income_median)) and age <= 60:
            employment = "unemployed"
        elif emp == 3:
            employment = "not in labor force"
        else:
            employment = "unknown"

        # Education
        education = df_nhs["education"].mode()[0] if "education" in df_nhs.columns and not df_nhs.empty else "Unknown"

        # Preparedness
        if "hrcn_prep_3cat" in df_nhs.columns and not df_nhs.empty:
            prep_counts = df_nhs["hrcn_prep_3cat"].value_counts()
            preparedness = prep_counts.idxmax() if not prep_counts.empty else "Unknown"
        else:
            preparedness = "Unknown"

        # Avg premium (with small variation for realism)
        try:
            avg_premium = (
                df_policy["Total Annual Payment"].sum() /
                df_policy["Policies in Force"].sum()
            )
        except (KeyError, ZeroDivisionError):
            avg_premium = 1500

        avg_premium = round(avg_premium)

        # Marital status
        if "MARST" in df_ipums.columns and not df_ipums.empty:
            m = df_ipums["MARST"].mode()[0]
            status = {
                1: "Married with spouse present",
                2: "Married with spouse absent",
                3: "Separated",
                4: "Divorced",
                5: "Widowed",
            }.get(m, "Single")
        else:
            status = "Unknown"

        # Number of kids
        kids = round(df_nhs["numchild"].mean()) if "numchild" in df_nhs.columns and not df_nhs.empty else 0

        # Flood data
        if not df_nfip.empty:
            row = df_nfip.sample(1).iloc[0]
            flood_zone = row.get("flood_zone", "Unknown")
            elevation_diff = row.get("elevation_difference_ft", "Unknown")
            flood_depth = row.get("water_depth_ft", "Unknown")
            construction_year = row.get("construction_year", "Unknown")
        else:
            flood_zone = elevation_diff = flood_depth = construction_year = "Unknown"

        # Hurricane data
        wind_avg = round(df_exp["max_wind_speed"].mean(),1) if not df_exp.empty else np.nan
        storm_count = df_exp["hurricane_name"].nunique() if not df_exp.empty else 0

        # Risk perception
        if "hrcn_risk_perception" in df_nhs.columns and not df_nhs.empty:
            risk_response = df_nhs["hrcn_risk_perception"].dropna().sample(1).iloc[0]
        else:
            risk_response = "Likely"

#------------------------------------Add in more random stuff for the genagent which is easier to make like so-----------------------------------------------#
    county_lookup = county_name.strip().lower()
    home_value = np.nan
    if not df_homeval.empty:
        df_homeval = df_homeval.copy()
        df_homeval["county"] = df_homeval["county"].str.strip().str.lower()
        match = df_homeval[df_homeval["county"] == county_lookup]

        if not match.empty:
            home_row = match.iloc[0]
            p25 = home_row.get("p25", np.nan)
            p50 = home_row.get("p50", np.nan)
            p75 = home_row.get("p75", np.nan)
            p95 = home_row.get("p95", np.nan)

            if mode == "min":
                home_value = p25
            elif mode == "max":
                home_value = p95
            elif mode == 'base':
                home_value = p50
            else:
                home_value = round(random.uniform(p25, p75))

    if "OWNERSHP" in df_ipums.columns and not df_ipums.empty:
        ownership_codes = {
            1: "own",
            2: "rent",
    }
    adults = df_ipums[df_ipums["AGE"] >= 18]
    if not adults.empty:
        if mode == "min":
            ownership_code = adults["OWNERSHP"].max()
        elif mode == "max":
            ownership_code = adults["OWNERSHP"].min()
        elif mode == "random":
            ownership_code = adults["OWNERSHP"].dropna().sample(1, replace=True).iloc[0]
        else:
            ownership_code = adults["OWNERSHP"].mode()[0]
        ownership = ownership_codes.get(int(ownership_code), "unknown")
    else:
        ownership = "unknown"


    # --- AGE DETERMINATION ---
    if not df_ipums.empty and "AGE" in df_ipums.columns:
    # Remove minors
        adults = df_ipums[df_ipums["AGE"] >= 18].copy()

    # Define bins and labels for age demographics
        bins = [0, 17, 25, 45, 65, 120]
        labels = ['Minor', 'Young Adult', 'Adult', 'Middle-Aged', 'Senior']

    # Assign age group
        adults["age_group"] = pd.cut(adults["AGE"], bins=bins, labels=labels, right=True)
        if not adults.empty:
            if mode == "min":
                age = int(adults["AGE"].min())
            elif mode == "max":
                age = int(adults["AGE"].max())
            elif mode == "random":
                age = int(adults["AGE"].sample(1).iloc[0])
            elif mode == "mean":
            # Pick the most common age group, then choose random age within that bin
                age_group = adults["age_group"].mode()[0]
                group_index = labels.index(age_group)
                low, high = bins[group_index] + 1, bins[group_index + 1]
                age = np.random.randint(low, high)
            else:
                age = int(adults["AGE"].median())


    persona = pd.DataFrame({
        "county": [county_name],
        "mode": [mode],
        "age": [age],
        "income": [income_median],
        "home_value": [home_value],
        # "premium": [avg_premium],
        "wind_avg": [wind_avg],
        "storm_count": [storm_count],
        "education": [education],
        "preparedness": [preparedness],
        "risk_response": [risk_response],
        "kids": [kids],
        "employment": [employment],
        "status": [status],
        "flood_zone": [flood_zone],
        "elevation_diff": [elevation_diff],
        "flood_depth": [flood_depth],
        "construction_year": [construction_year],
        "ownership": ownership
    })

    return persona

In [ ]:
counties = ["Duval", "Broward", "Alachua", "St. Johns", "Bay", 'Leon', "Brevard", "Pasco", "Seminole",
            "Palm Beach", "Osceola", "Hillsborough", "Pinellas", "Manatee", "Sarasota", "Lee"]
nearby_map = {
    "Duval": ["St. Johns", "Clay", "Nassau", "Baker"],
    "St. Johns": ["Duval", "Flagler", "Putnam", "Clay"],
    "Alachua": ["Levy", "Marion", "Columbia", "Gilchrist"],
    "Leon": ["Gadsden", "Wakulla", "Jefferson", "Liberty"],
    "Bay": ["Gulf", "Washington", "Calhoun", "Walton"],
    "Broward": ["Miami-Dade", "Palm Beach", "Collier"],
    "Palm Beach": ["Broward", "Martin", "Okeechobee", "Hendry"],
    "Brevard": ["Orange", "Volusia", "Indian River", "Osceola"],
    "Seminole": ["Orange", "Volusia", "Lake", "Brevard"],
    "Osceola": ["Orange", "Polk", "Brevard", "Lake"],
    "Hillsborough": ["Pasco", "Pinellas", "Polk", "Manatee"],
    "Pinellas": ["Hillsborough", "Manatee", "Pasco"],
    "Pasco": ["Hernando", "Pinellas", "Hillsborough", "Polk"],
    "Manatee": ["Sarasota", "Hillsborough", "Pinellas", "Hardee"],
    "Sarasota": ["Manatee", "Charlotte", "DeSoto", "Lee"],
    "Lee": ["Charlotte", "Collier", "Hendry", "Glades"],
}

personas = []
county_persona_amount = 3  # 1 mean + 2 random
np.random.seed(123)
for c in counties:
    nearby = nearby_map.get(c, [])

    # Only ONE mean persona per county (base line)
    persona_mean = build_persona(c, nearby=nearby_map, mode="base")
    personas.append(persona_mean)

     # Generate 1 max persona
    persona_max = build_persona(c, nearby=nearby_map, mode="max")
    personas.append(persona_max)

    # Generate 1 min persona
    persona_min = build_persona(c, nearby=nearby_map, mode="min")
    personas.append(persona_min)

    # this makes the randomness in the data
    for i in range(county_persona_amount - 1):
        persona_random = build_persona(c, nearby=nearby_map, mode="random")
        personas.append(persona_random)

# Combine into one DataFrame
personas_df = pd.concat(personas, ignore_index=True)

# random issue with concat so had to make a function that removes the duplicates that cause errors
def deduplicate_columns(columns):
    counts = {}
    new_columns = []
    for col in columns:
        if col not in counts:
            counts[col] = 0
            new_columns.append(col)
        else:
            counts[col] += 1
            new_columns.append(f"{col}.{counts[col]}")
    return new_columns

personas_df.columns = deduplicate_columns(personas_df.columns)

personas_df.to_csv("minmax.csv", index=False)
personas_df.to_json("minmax.json", orient="records", indent=4)
print("\n✅ Saved personas to both CSV and JSON files.")


📊 Building persona for Duval (FIPS suffix 031)
Home: Using Duval County data.
Exposure: Using Duval County data.
NHS: No Duval data — using St. Johns County instead.
HomeVal: Using Duval County data.

📊 Building persona for Duval (FIPS suffix 031)
Home: Using Duval County data.
Exposure: Using Duval County data.
NHS: No Duval data — using St. Johns County instead.
HomeVal: Using Duval County data.

📊 Building persona for Duval (FIPS suffix 031)
Home: Using Duval County data.
Exposure: Using Duval County data.
NHS: No Duval data — using St. Johns County instead.
HomeVal: Using Duval County data.

📊 Building persona for Duval (FIPS suffix 031)
Home: Using Duval County data.
Exposure: Using Duval County data.
NHS: No Duval data — using St. Johns County instead.
HomeVal: Using Duval County data.

📊 Building persona for Duval (FIPS suffix 031)
Home: Using Duval County data.
Exposure: Using Duval County data.
NHS: No Duval data — using St. Johns County instead.
HomeVal: Using Duval County d